In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
import warnings
from tqdm import tqdm
import os, sys
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('train.csv')

In [17]:
path = os.getcwd()

In [21]:
img = []
for i in tqdm(range(len(data))):
    temp_img = Image.open(path+data['img_path'].iloc[i][1:])
    temp_img = np.array(temp_img.resize((128,128)))
    img.append(temp_img)

100%|████████████████████████████████████████████████████████████████████████████| 16986/16986 [03:31<00:00, 80.49it/s]


In [22]:
cate = data['cat3'].unique()
cate_dic = {}
reverse_cate = {}
for i in range(len(cate)):
    cate_dic[cate[i]] = i
    reverse_cate[i] = cate[i]

In [23]:
train_input = np.array(img)
train_target = data['cat3'].apply(lambda x : cate_dic[x])

In [24]:
inputs = tf.keras.layers.Input(shape=(128,128,3,))

conv_1 = tf.keras.layers.Conv2D(filters=4, kernel_size=3, padding='same', activation='relu')(inputs)
maxpool_1 = tf.keras.layers.MaxPooling2D(pool_size=2, padding='same')(conv_1)
dropout_1 = tf.keras.layers.Dropout(0.2)(maxpool_1)

conv_2 = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu')(dropout_1)
maxpool_2 = tf.keras.layers.MaxPooling2D(pool_size=2, padding='same')(conv_2)
dropout_2 = tf.keras.layers.Dropout(0.2)(maxpool_2)

conv_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(dropout_2)
maxpool_3 = tf.keras.layers.MaxPooling2D(pool_size=2, padding='same')(conv_3)
dropout_3 = tf.keras.layers.Dropout(0.2)(maxpool_3)

flatten = tf.keras.layers.Flatten()(dropout_3)
dense = tf.keras.layers.Dense(512, activation='relu')(flatten)

outputs = tf.keras.layers.Dense(len(cate_dic), activation='softmax')(dense)

In [25]:
model = tf.keras.models.Model(inputs, outputs)

In [26]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [34]:
model.fit(train_input, train_target, epochs=10)

Epoch 1/10
531/531 [==============================] - 73s 138ms/step - loss: 1.0158 - acc: 0.7253
Epoch 2/10
531/531 [==============================] - 76s 143ms/step - loss: 0.8959 - acc: 0.7597
Epoch 3/10
531/531 [==============================] - 77s 145ms/step - loss: 0.8213 - acc: 0.7792
Epoch 4/10
531/531 [==============================] - 77s 145ms/step - loss: 0.7325 - acc: 0.8081
Epoch 5/10
531/531 [==============================] - 77s 146ms/step - loss: 0.6749 - acc: 0.8183
Epoch 6/10
531/531 [==============================] - 79s 149ms/step - loss: 0.6197 - acc: 0.8337
Epoch 7/10
531/531 [==============================] - 79s 148ms/step - loss: 0.5631 - acc: 0.8493
Epoch 8/10
531/531 [==============================] - 79s 148ms/step - loss: 0.4842 - acc: 0.8701
Epoch 9/10
531/531 [==============================] - 78s 147ms/step - loss: 0.4809 - acc: 0.8723
Epoch 10/10
531/531 [==============================] - 79s 148ms/step - loss: 0.4347 - acc: 0.8820


In [35]:
test_data = pd.read_csv('test.csv')

In [41]:
test_img = []
for i in tqdm(range(len(test_data))):
    temp_img = Image.open(path+test_data['img_path'].iloc[i][1:])
    temp_img = np.array(temp_img.resize((128,128)))
    test_img.append(temp_img)

 43%|█████████████████████████████████                                            | 7280/16986 [01:29<01:59, 81.14it/s]


IndexError: single positional indexer is out-of-bounds

In [42]:
result = model.predict(np.array(test_img))

In [47]:
result_se = pd.Series(np.argmax(result, axis=1)).apply(lambda x:reverse_cate[x])

In [48]:
submit_df = pd.read_csv('sample_submission.csv')

In [49]:
submit_df['cat3'] = result_se

In [54]:
submit_df.to_csv('submit_sec.csv', index=False)